In [1]:
import pandas as pd
import re

Funciones

In [2]:
def duraciones_a_lista(texto):
    if pd.isna(texto):
        return []
    
    # Extrae cada set con duración
    sets = re.findall(r'(\d{1,2}(?:st|nd|rd|th) set • (?:\d+h\s*)?\d+m)', texto)

    def set_num(set_str):
        return int(re.match(r'(\d+)', set_str).group(1))
    
    def duracion_en_minutos(set_str):
        duracion = re.search(r'((\d+)h)?\s*(\d+)m', set_str)
        horas = int(duracion.group(2)) if duracion.group(2) else 0
        minutos = int(duracion.group(3))
        return horas * 60 + minutos

    # Ordenar los sets (por número)
    sets_ordenados = sorted(sets, key=set_num)
    
    # Convertir a lista de minutos
    return [duracion_en_minutos(s) for s in sets_ordenados]

def convertir_a_minutos(duracion):
    if pd.isna(duracion):  # Si es NaN
        return None
    duracion = duracion.strip().lower()
    horas = 0
    minutos = 0
    if 'h' in duracion:
        partes = duracion.split('h')
        horas = int(partes[0].strip())
        if 'm' in partes[1]:
            minutos = int(partes[1].replace('m', '').strip())
    elif 'm' in duracion:
        minutos = int(duracion.replace('m', '').strip())
    return horas * 60 + minutos

def limpiar_sets(x):
    try:
        # Intenta convertir a número
        valor = float(x)
        # Si es 0, 1, 2 o 3 exactos, lo convertimos a int
        if valor in [0, 1, 2, 3]:
            return int(valor)
        else:
            return x  # No convertir si es otro número
    except:
        return x  # Dejar el valor original si no se puede convertir

ANÁLISIS

In [3]:
datos1 = pd.read_csv("data\\partidos_tenis_05_26-06_04.csv")
datos2 = pd.read_csv("data\\partidos_tenis_05_25.csv")
datos3 = pd.read_csv("data\\partidos_tenis_05_24.csv")
datos4 = pd.read_csv("data\\partidos_tenis_05_23.csv")
datos5 = pd.read_csv("data\\partidos_tenis_05_22-05_19.csv")
datos6 = pd.read_csv("data\\partidos_tenis_05_18-05_11.csv")
datos7 = pd.read_csv("data\\partidos_tenis_05_10-05_01.csv")
datos8 = pd.read_csv("data\\partidos_tenis_04_30-04_28.csv")
datos9 = pd.read_csv("data\\partidos_tenis_04_27-04_25.csv")
datos10 = pd.read_csv("data\\partidos_tenis_04_24-04_22.csv")
datos11 = pd.read_csv("data\\partidos_tenis_04_21-04_17.csv")
datos12 = pd.read_csv("data\\partidos_tenis_04_17-04_14.csv")
datos13 = pd.read_csv("data\\partidos_tenis_04_13-04_09.csv")
datos14 = pd.read_csv("data\\partidos_tenis_04_08-03_30.csv")
datos15 = pd.read_csv("data\\partidos_tenis_03_29-03_20.csv")
datos16 = pd.read_csv("data\\partidos_tenis_03_20-03_19.csv")
datos17 = pd.read_csv("data\\partidos_tenis_03_18-03_17.csv")
datos18 = pd.read_csv("data\\partidos_tenis_03_16-03_14.csv")
datos19 = pd.read_csv("data\\partidos_tenis_03_13-03_11.csv")

dfs = [datos1, datos2, datos3, datos4, datos5, datos6, datos7, datos8, datos9, datos10, datos11, datos12, datos13, datos14, datos15, datos16, datos17, datos18, datos19]
raw = pd.concat(dfs, ignore_index=True)
raw = raw.drop_duplicates()
raw.drop("cancelado", axis = 1, inplace = True)
raw['setsA'] = raw['setsA'].apply(limpiar_sets)
raw['setsB'] = raw['setsB'].apply(limpiar_sets)

raw['cancelado'] = (~raw['setsA'].apply(lambda x: isinstance(x, int))) | (~raw['setsB'].apply(lambda x: isinstance(x, int)))

datos = raw[~raw['cancelado']].copy()
datos["cancelado"] = datos["cancelado"].astype(int)
datos["genero"] = datos["genero"].map({"masculino": 1, "femenino": 0}).astype("Int64")


Separo los datos entre individual y dobles

In [156]:
filtroDobles = (datos['especificacion_torneo'] == 'Doubles') | (datos['especificacion_torneo'] == 'Mixed Doubles')
filtroIndividual = (datos['especificacion_torneo'] == 'Singles') | (datos['especificacion_torneo'] == 'Qualifying')

dob = datos[filtroDobles].copy()
ind = datos[filtroIndividual].copy()

print("Dobles",len(dob))
print("Individuales",len(ind))
print("Todo",len(datos))

Dobles 192
Individuales 1465
Todo 1657


Me especializo en individual

In [157]:
ind.drop(["jugadorA2", "jugadorB2", "nombre_torneo", "duracion"], axis = 1, inplace = True)

In [158]:
ind[["retirado", "duracion"]] = ind["duracion_sets"].str.split(" - ", expand=True)

In [ ]:
ind['retirado'] = ind['retirado'].replace('Final', 'FT')

255     None
276     None
761     None
865     None
874     None
896     None
951     None
964     None
997     None
1052    None
1086    None
1566    None
Name: duracion, dtype: object